In [102]:
import pandas as pd

In [103]:
def categorize_hl(row):
    if row <= 7200:
        return 'Low'
    elif 7200 < row <= 18000:
        return 'Medium'
    else:
        return 'High'

In [104]:
def float_or_false(value):
    try:
        if float(value):
            return True
    except ValueError:
        return False

In [105]:
def mod_false(df):
    for i in range(len(df)):
        if (
            df.loc[i, 'N-terminal_Modification'] != 'Free' or
            df.loc[i, 'C-terminal_Modification'] != 'Free' or
            df.loc[i, 'Linear_Cyclic'] != 'Linear' or
            df.loc[i, 'Other_Modification'] not in [None, 'None', '']
        ):         
            df.loc[i,'is_mod'] = 'True'
            df.loc[i, 'modifications'] = f"{df.loc[i, 'N-terminal_Modification']}, {df.loc[i, 'C-terminal_Modification']}, {'Cyclic' if df.loc[i, 'Linear_Cyclic'] == 'Cyclic' else 'Linear'}, {df.loc[i, 'Other_Modification']}"
        else:
            df.loc[i,'is_mod'] = 'False'
            df.loc[i,'modifications'] = ''

In [106]:
df = pd.read_csv('antiviral_original.csv')

In [107]:
df2 = df
mod_false(df2)
df2.drop(['N-terminal_Modification', 'C-terminal_Modification'], axis=1, inplace=True)
df2['modifications'] = df2['modifications'].replace([', Free', 'Free,'], '', regex=True)
df2 = df2.rename(columns={'Sequence': 'sequence'})
df2['sequence'] = df2['sequence'].str.upper()

In [108]:
df2

,sequence,Source,Target_Organism,Assay,Linear_Cyclic,Other_Modification,Binding_Target,experimental_characteristics,Half_Life,is_mod,modifications
0,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,None,autolysis,E_coli,>10 hour,False,
1,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,None,autolysis,Mammalian,4.4 hour,False,
2,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,None,autolysis,Yeast,>20 hour,False,
3,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Plaque formation assay,Linear,None,membrane,E_coli,>10 hour,False,
4,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,None,cell membrane,Mammalian,4.4 hour,False,
...,...,...,...,...,...,...,...,...,...,...,...
5953,YAGAVVNDL,Synthetic construct,HSV,Ribonucleotide reductase assay,Linear,None,ribonucleotide reductase,Mammalian,NaN,True,"Acetylation, Linear, None"
5954,YAGAVVNDL,Synthetic construct,HSV,Ribonucleotide reductase assay,Linear,None,ribonucleotide reductase,E_coli,NaN,False,
5955,YAIIXYNKYXNC,Synthetic construct,"ZIKV,DENV,WNV",protease enzymatic inhibition assay,Linear,The 'X' at position 5 indicates N-methyl-4-O-m...,NS2B-NS3 protease,Yeast,NaN,True,Thioether-acyl moiety (−S-Ac−)(forms a macrocy...
5956,YAIIXYNKYXNC,Synthetic construct,"ZIKV,DENV,WNV",protease enzymatic inhibition assay,Linear,The 'X' at position 5 indicates N-methyl-4-O-m...,NS2B-NS3 protease,E_coli,NaN,True,Thioether-acyl moiety (−S-Ac−)(forms a macrocy...


In [109]:
#dimensiones de datos duplicados
df2[df2.duplicated(subset=['sequence', 'experimental_characteristics'], keep=False)].shape

(777, 11)

In [110]:
#dimensiones de datos no duplicados
df2[~df2.duplicated(subset=['sequence', 'experimental_characteristics'], keep=False)].shape

(5181, 11)

In [111]:
#separacion de datos duplicados que tengan la misma secuencia y modificaciones (se examinan las secuencia, si modificaiones y secuencia es igual pero half_life_seconds es diferente se borran todos los duplicados)
df_data_duplicated=(df2[df2.duplicated(subset=['sequence', 'experimental_characteristics'], keep=False)]).sort_values(by='sequence')
df_data_duplicated

,sequence,Source,Target_Organism,Assay,Linear_Cyclic,Other_Modification,Binding_Target,experimental_characteristics,Half_Life,is_mod,modifications
3,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Plaque formation assay,Linear,None,membrane,E_coli,>10 hour,False,
4,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,None,cell membrane,Mammalian,4.4 hour,False,
5,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,None,cell membrane,Yeast,>20 hour,False,
6,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,None,cell membrane,E_coli,>10 hour,False,
7,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Plaque formation assay,Linear,None,membrane,Mammalian,4.4 hour,False,
...,...,...,...,...,...,...,...,...,...,...,...
5815,YTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWF,Synthetic construct,"HIV,SIV","MAGI/cMAGI infectivity assay,neutralization assay",Linear,None,membrane,Yeast,10 min,True,"Acetylation, Amidation, Linear, None"
5816,YTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWF,Synthetic construct,"HIV,SIV","MAGI/cMAGI infectivity assay,neutralization assay",Linear,None,membrane,E_coli,2 min,True,"Acetylation, Amidation, Linear, None"
5817,YTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWF,Synthetic construct,HIV,"MAGI/cMAGI infectivity assay,neutralization assay",Linear,None,membrane,Mammalian,2.8 hour,False,
5818,YTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWF,Synthetic construct,"HIV,SIV","MAGI/cMAGI infectivity assay,neutralization assay",Linear,None,membrane,Mammalian,2.8 hour,True,"Acetylation, Amidation, Linear, None"


In [112]:
#separacion de datos duplicados que tengan la misma secuencia, modificaciones y half_life_seconds (de esto solo se guarda una copia) 
df_data_duplicated_2=(df2[df2.duplicated(subset=['sequence', 'Half_Life', 'experimental_characteristics'], keep=False)]).sort_values(by='sequence')
df_data_duplicated_2

,sequence,Source,Target_Organism,Assay,Linear_Cyclic,Other_Modification,Binding_Target,experimental_characteristics,Half_Life,is_mod,modifications
3,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Plaque formation assay,Linear,None,membrane,E_coli,>10 hour,False,
4,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,None,cell membrane,Mammalian,4.4 hour,False,
5,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,None,cell membrane,Yeast,>20 hour,False,
6,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,None,cell membrane,E_coli,>10 hour,False,
7,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Plaque formation assay,Linear,None,membrane,Mammalian,4.4 hour,False,
...,...,...,...,...,...,...,...,...,...,...,...
5816,YTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWF,Synthetic construct,"HIV,SIV","MAGI/cMAGI infectivity assay,neutralization assay",Linear,None,membrane,E_coli,2 min,True,"Acetylation, Amidation, Linear, None"
5815,YTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWF,Synthetic construct,"HIV,SIV","MAGI/cMAGI infectivity assay,neutralization assay",Linear,None,membrane,Yeast,10 min,True,"Acetylation, Amidation, Linear, None"
5814,YTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWF,Synthetic construct,HIV,"MAGI/cMAGI infectivity assay,neutralization assay",Linear,None,membrane,E_coli,2 min,False,
5819,YTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWF,Synthetic construct,HIV,"MAGI/cMAGI infectivity assay,neutralization assay",Linear,None,membrane,Yeast,10 min,False,


In [113]:
#separacion de datos no duplicados
df_not_duplicated=(df2[~df2.duplicated(subset=['sequence', 'experimental_characteristics'], keep=False)]).sort_values(by='sequence')
#se concatenan los datos no duplicados con los datos duplicados pero que se mantiene una copia
#Como quito los datos duplicados que tiene casi todo igual
df_not_duplicated=pd.concat([df_not_duplicated, df_data_duplicated_2])
df_not_duplicated

,sequence,Source,Target_Organism,Assay,Linear_Cyclic,Other_Modification,Binding_Target,experimental_characteristics,Half_Life,is_mod,modifications
0,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,None,autolysis,E_coli,>10 hour,False,
1,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,None,autolysis,Mammalian,4.4 hour,False,
2,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,None,autolysis,Yeast,>20 hour,False,
9,AAGAVVNDL,Synthetic construct,HSV,Ribonucleotide reductase assay,Linear,None,ribonucleotide reductase,Yeast,>20 hour,False,
10,AAGAVVNDL,Synthetic construct,HSV,Ribonucleotide reductase assay,Linear,None,ribonucleotide reductase,Mammalian,4.4 hour,False,
...,...,...,...,...,...,...,...,...,...,...,...
5816,YTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWF,Synthetic construct,"HIV,SIV","MAGI/cMAGI infectivity assay,neutralization assay",Linear,None,membrane,E_coli,2 min,True,"Acetylation, Amidation, Linear, None"
5815,YTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWF,Synthetic construct,"HIV,SIV","MAGI/cMAGI infectivity assay,neutralization assay",Linear,None,membrane,Yeast,10 min,True,"Acetylation, Amidation, Linear, None"
5814,YTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWF,Synthetic construct,HIV,"MAGI/cMAGI infectivity assay,neutralization assay",Linear,None,membrane,E_coli,2 min,False,
5819,YTSLIHSLIEESQNQQEKNEQELLELDKWASLWNWF,Synthetic construct,HIV,"MAGI/cMAGI infectivity assay,neutralization assay",Linear,None,membrane,Yeast,10 min,False,


In [114]:
#se borran valores nulos y se eliminan los duplicados sobrantes de los que se guarda una copia y se ordenan los datos
df_not_duplicated.dropna(subset=['sequence', 'Half_Life'], inplace=True)
df_not_duplicated.drop_duplicates(subset=['sequence', 'experimental_characteristics'], keep='first', inplace=True)
df_not_duplicated.sort_index(inplace=True)

In [115]:
df_not_duplicated

,sequence,Source,Target_Organism,Assay,Linear_Cyclic,Other_Modification,Binding_Target,experimental_characteristics,Half_Life,is_mod,modifications
0,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,None,autolysis,E_coli,>10 hour,False,
1,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,None,autolysis,Mammalian,4.4 hour,False,
2,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,None,autolysis,Yeast,>20 hour,False,
3,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Plaque formation assay,Linear,None,membrane,E_coli,>10 hour,False,
4,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,None,cell membrane,Mammalian,4.4 hour,False,
...,...,...,...,...,...,...,...,...,...,...,...
5881,YXKXKXXKXXKC,Synthetic construct,"ZIKV,DENV,WNV",protease enzymatic inhibition assay,Linear,"The 'X' at position 2,7,9 and 10 indicates MeY...",NS2B-NS3 protease,E_coli,2 min,True,Thioether-acyl moiety (−S-Ac−)(forms a macrocy...
5882,YXKXKXXKXXKC,Synthetic construct,"ZIKV,DENV,WNV",protease enzymatic inhibition assay,Linear,"The 'X' at position 2,7,9 and 10 indicates MeY...",NS2B-NS3 protease,Yeast,10 min,True,Thioether-acyl moiety (−S-Ac−)(forms a macrocy...
5885,YAGAVVNDL,Synthetic construct,HSV,Ribonucleotide reductase assay,Linear,None,ribonucleotide reductase,Yeast,10 min,True,"Acetylation, Linear, None"
5886,YAGAVVNDL,Synthetic construct,HSV,Ribonucleotide reductase assay,Linear,None,ribonucleotide reductase,E_coli,2 min,False,


In [116]:
df_float=df_not_duplicated[~df_not_duplicated['Half_Life'].str.startswith('>', '<')]
df_object=df_not_duplicated[df_not_duplicated['Half_Life'].str.startswith('>', '<')]

In [117]:
df_float = df_float[df_float['Half_Life'].str.contains(r'^\d+\.?\d*\s*(hour|min)$')]
df_float.reset_index(drop=True, inplace=True)
for i in range(len(df_float)):
    if 'hour' in df_float.loc[i, 'Half_Life']:
        df_float.loc[i, 'Half_Life'] = float(df_float.loc[i, 'Half_Life'].split(' ')[0]) * 3600
    else:
        df_float.loc[i, 'Half_Life'] = float(df_float.loc[i, 'Half_Life'].split(' ')[0]) * 60

C:\Users\Javi\AppData\Local\Temp\ipykernel_13240\1304014241.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_float = df_float[df_float['Half_Life'].str.contains(r'^\d+\.?\d*\s*(hour|min)$')]


In [118]:
df_float.reset_index(drop=True, inplace=True)
df_object.reset_index(drop=True, inplace=True)

In [119]:
df_float

,sequence,Source,Target_Organism,Assay,Linear_Cyclic,Other_Modification,Binding_Target,experimental_characteristics,Half_Life,is_mod,modifications
0,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,None,autolysis,Mammalian,15840.0,False,
1,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,None,cell membrane,Mammalian,15840.0,False,
2,AAGAVVNDL,Synthetic construct,HSV,Ribonucleotide reductase assay,Linear,None,ribonucleotide reductase,Mammalian,15840.0,False,
3,AAHLIDALYAEFLGGRVLTT,Synthetic construct(derived from HSV-1 H glyco...,HSV,Plaque assay,Linear,None,mambrane,Mammalian,15840.0,False,
4,AAHLIDALYAEFLGGRVLTTPVVHRALFYASAVLRQPFLAGVPSA,Synthetic construct(derived from HSV-1 H glyco...,HSV,Plaque assay,Linear,None,mambrane,Mammalian,15840.0,False,
...,...,...,...,...,...,...,...,...,...,...,...
3481,YXKXKXXKXXKC,Synthetic construct,"ZIKV,DENV,WNV",protease enzymatic inhibition assay,Linear,"The 'X' at position 2,7,9 and 10 indicates MeY...",NS2B-NS3 protease,E_coli,120.0,True,Thioether-acyl moiety (−S-Ac−)(forms a macrocy...
3482,YXKXKXXKXXKC,Synthetic construct,"ZIKV,DENV,WNV",protease enzymatic inhibition assay,Linear,"The 'X' at position 2,7,9 and 10 indicates MeY...",NS2B-NS3 protease,Yeast,600.0,True,Thioether-acyl moiety (−S-Ac−)(forms a macrocy...
3483,YAGAVVNDL,Synthetic construct,HSV,Ribonucleotide reductase assay,Linear,None,ribonucleotide reductase,Yeast,600.0,True,"Acetylation, Linear, None"
3484,YAGAVVNDL,Synthetic construct,HSV,Ribonucleotide reductase assay,Linear,None,ribonucleotide reductase,E_coli,120.0,False,


In [120]:
df_not_duplicated

,sequence,Source,Target_Organism,Assay,Linear_Cyclic,Other_Modification,Binding_Target,experimental_characteristics,Half_Life,is_mod,modifications
0,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,None,autolysis,E_coli,>10 hour,False,
1,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,None,autolysis,Mammalian,4.4 hour,False,
2,AAAMSQVTN,Synthetic construct,HIV,Protease inhibition assay,Linear,None,autolysis,Yeast,>20 hour,False,
3,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct(derived from SARS-CoV spik...,SARS-CoV,Plaque formation assay,Linear,None,membrane,E_coli,>10 hour,False,
4,AACEVAKNLNESLIDLQELGKYEQYIKW,Synthetic construct,SARS-CoV,Plaque reduction assay,Linear,None,cell membrane,Mammalian,4.4 hour,False,
...,...,...,...,...,...,...,...,...,...,...,...
5881,YXKXKXXKXXKC,Synthetic construct,"ZIKV,DENV,WNV",protease enzymatic inhibition assay,Linear,"The 'X' at position 2,7,9 and 10 indicates MeY...",NS2B-NS3 protease,E_coli,2 min,True,Thioether-acyl moiety (−S-Ac−)(forms a macrocy...
5882,YXKXKXXKXXKC,Synthetic construct,"ZIKV,DENV,WNV",protease enzymatic inhibition assay,Linear,"The 'X' at position 2,7,9 and 10 indicates MeY...",NS2B-NS3 protease,Yeast,10 min,True,Thioether-acyl moiety (−S-Ac−)(forms a macrocy...
5885,YAGAVVNDL,Synthetic construct,HSV,Ribonucleotide reductase assay,Linear,None,ribonucleotide reductase,Yeast,10 min,True,"Acetylation, Linear, None"
5886,YAGAVVNDL,Synthetic construct,HSV,Ribonucleotide reductase assay,Linear,None,ribonucleotide reductase,E_coli,2 min,False,


In [121]:
df_float.rename(columns={'Half_Life': 'half_life_seconds'}, inplace=True)
df_object.rename(columns={'Half_Life': 'half_life'}, inplace=True)

C:\Users\Javi\AppData\Local\Temp\ipykernel_13240\905327210.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_object.rename(columns={'Half_Life': 'half_life'}, inplace=True)


In [122]:
df_float=df_float.filter(['sequence', 'half_life_seconds', 'is_mod', 'modifications', 'experimental_characteristics'])
df_object=df_object.filter(['sequence', 'half_life', 'is_mod', 'modifications', 'experimental_characteristics'])

In [123]:
df_float['hl_category'] = df_float['half_life_seconds'].apply(categorize_hl)

In [124]:
df_float

,sequence,half_life_seconds,is_mod,modifications,experimental_characteristics,hl_category
0,AAAMSQVTN,15840.0,False,,Mammalian,Medium
1,AACEVAKNLNESLIDLQELGKYEQYIKW,15840.0,False,,Mammalian,Medium
2,AAGAVVNDL,15840.0,False,,Mammalian,Medium
3,AAHLIDALYAEFLGGRVLTT,15840.0,False,,Mammalian,Medium
4,AAHLIDALYAEFLGGRVLTTPVVHRALFYASAVLRQPFLAGVPSA,15840.0,False,,Mammalian,Medium
...,...,...,...,...,...,...
3481,YXKXKXXKXXKC,120.0,True,Thioether-acyl moiety (−S-Ac−)(forms a macrocy...,E_coli,Low
3482,YXKXKXXKXXKC,600.0,True,Thioether-acyl moiety (−S-Ac−)(forms a macrocy...,Yeast,Low
3483,YAGAVVNDL,600.0,True,"Acetylation, Linear, None",Yeast,Low
3484,YAGAVVNDL,120.0,False,,E_coli,Low


In [125]:
df_object

,sequence,half_life,is_mod,modifications,experimental_characteristics
0,AAAMSQVTN,>10 hour,False,,E_coli
1,AAAMSQVTN,>20 hour,False,,Yeast
2,AACEVAKNLNESLIDLQELGKYEQYIKW,>10 hour,False,,E_coli
3,AACEVAKNLNESLIDLQELGKYEQYIKW,>20 hour,False,,Yeast
4,AAGAVVNDL,>20 hour,False,,Yeast
...,...,...,...,...,...
1871,VVTLTDTTNQKTELQAIYLA,>20 hour,False,,Yeast
1872,VYLHRIDLGPPISLERLDVGTNLQNAIAKLEDAKE,>10 hour,True,"Acetylation, Amidation, Linear, None",E_coli
1873,VYLHRIDLGPPISLERLDVGTNLQNAIAKLEDAKE,>20 hour,True,"Acetylation, Amidation, Linear, None",Yeast
1874,VYTDKVDISSQISSMNQSLQQSKDYIKEAQKILDTV,>20 hour,False,,Yeast


In [126]:
#Por si se quieren quitar las modificaciones
df_filtered = df_float[df_float['is_mod'] == 'False']
df_filtered = df_filtered.drop(columns=['is_mod', 'modifications'])
df_filtered_ob = df_object[df_object['is_mod'] == 'False']
df_filtered_ob = df_filtered_ob.drop(columns=['is_mod', 'modifications'])

In [127]:
df_float.to_csv('antiviral.csv', index=False)
df_filtered.to_csv('antiviral.csv', index=False)
df_object.to_csv('antiviral_object.csv', index=False)